# Concatenating Data

In this chapter, we focus on concatenating two or more Series or DataFrames together. Concatenating data in pandas refers to stacking DataFrames or Series vertically one on top of the other, or horizontally, side by side.

## Concatenation with the `pd.concat` function

The `pd.concat` function (NOT a method) is flexible and versatile with multiple arguments to give you power to combine two or more Series or DataFrames. Let's read in two DataFrames containing a few rows of stock price data on Amazon (AMZN) and Apple (AAPL), each with two columns (after setting the index to be the date) to help illustrate how the `pd.concat` function works.

In [ ]:
import pandas as pd
amzn = pd.read_csv('../data/stocks/amzn_sample.csv', 
                   parse_dates=['date'], index_col='date')
aapl = pd.read_csv('../data/stocks/aapl_sample.csv', 
                   parse_dates=['date'], index_col='date')
amzn

In [ ]:
aapl

### Stacking DataFrames vertically one on top of the other

We'll begin by stacking these DataFrames vertically one on top of the other resulting in a single new DataFrame. The first and only required argument to `pd.concat` must be a sequence of Series/DataFrames. By default, they will be stacked vertically. Here, we use a list to pass in the two above DataFrames.

In [ ]:
pd.concat([amzn, aapl])

Notice that the index from each original DataFrame was kept. Set `ignore_index` to `True` to discard the old index and to use a completely new `RangeIndex` beginning from 0.

In [ ]:
pd.concat([amzn, aapl], ignore_index=True)

### Label each DataFrame with the `keys` parameter

Use the `keys` parameter to label each original DataFrame. This label will be placed in the outermost level of the index always resulting in a multi-level index. Pass the `names` parameter a list of strings to give each index level a name. When passing a single item list, the string will be used just for the new outermost level.

In [ ]:
df = pd.concat([amzn, aapl], keys=['amzn', 'aapl'], names=['symbol'])
df

Because a multi-level index can be difficult to use, I suggest calling the `reset_index` method to convert each of the index levels to columns.

In [ ]:
df.reset_index()

### Alternative way to avoid a multi-level index

Instead of using the `keys` and `names` parameter to label the sections of data resulting from the `pd.concat` function, we could have created a new column in our original DataFrames, first.

In [ ]:
amzn['symbol'] = 'amzn'
aapl['symbol'] = 'aapl'
pd.concat([amzn, aapl])

## Automatic alignment of index always takes place

As is the case with all interactions involving two Series/DataFrames, automatic alignment of the index happens first. When concatenating vertically, the columns align. For instance, let's rename the second column of the `amzn` DataFrame to `adjusted_close` and call `pd.concat` again.

In [ ]:
amzn2 = amzn.rename(columns={'adjusted_close': 'close'})
df = pd.concat([amzn2, aapl], ignore_index=True)
df.style.highlight_null(null_color='tan')

Automatic alignment of the columns happens first when concatenating vertically. The missing values are highlighted where there is misalignment. By default, an 'outer join' occurs, where columns unique to each DataFrame will always appear in the result, but with missing values. To force `pd.concat` to only keep columns that are members of both DataFrames, set the `join` parameter to `'inner'`. Here, no closing price column remains in the result.

In [ ]:
pd.concat([amzn2, aapl], join='inner')

## Change the direction of concatenation with `axis`

As with most pandas operation, `pd.concat` has the `axis` parameter available to change the direction of the concatenation to horizontal. Now, the automatic alignment takes place on the indexes of each DataFrame.

In [ ]:
pd.concat([amzn, aapl], axis=1)

The `keys` parameter continues to label each of the original DataFrame, this time creating a multi-level column index in the process.

In [ ]:
pd.concat([amzn, aapl], axis=1, keys=['AMZN', 'AAPL'])

## Concatenating many DataFrames together

It's possible to concatenate any number of DataFrames together. The `olympic` folder contains CSV files of all medals won by country for each Summer Olympics. The name of each file is the year the Olympics was hosted beginning in 1896. The `pathlib` [standard library][0] provides the excellent `Path` class which allows us to collect all of the file names in a particular directory with its `glob` method. Each `Path` object has a `stem` attribute which is the name of the of the file without the file extension and in our case will just be the year. Below, we iterate through each file reading in each medal table as a DataFrame. A dictionary with keys as the integer year are used to store the DataFrames.

[0]: https://docs.python.org/3/library/pathlib.html

In [ ]:
from pathlib import Path
olympic_data_path = Path('../data/olympics')
paths = sorted(olympic_data_path.glob('*.csv'))
dfs = {}
for path in paths:
    year = int(path.stem)
    dfs[year] = pd.read_csv(path)

All nations from the first modern Olympics are output below.

This dictionary of DataFrames may be passed to the `pd.concat` function, and when doing so, the keys automatically get placed in the index to label the resulting data. Labeling this index level is still possible with the `names` parameter.

In [ ]:
df_medals = pd.concat(dfs, names=['year'])
df_medals.head()

### Dropping an index level

When concatenating DataFrames, the original index is kept unless setting `ignore_index` to `False`. Notice above how the original index for each DataFrame is kept. It is subtle as there is no level name for it, but it is directly to the right of the year level. Levels from both the index and columns may be removed with the `droplevel` method. Pass it the name or integer location of the level name to drop.

In [ ]:
df_medals = df_medals.droplevel(1)
df_medals.tail()

The number of unique teams competing per year can be found by grouping the index column and counting the number of rows.

In [ ]:
df_medals.groupby('year').size().tail()

We verify these results by iterating through our dictionary of DataFrames and printing out the number of rows of eeach for the last five years of data.

In [ ]:
for year, df in dfs.items():
    if year >= 2004:
        print(year, len(df))

## Concatenating Series together

Series may also be concatenated together with the `pd.concat` function. Below, we create two simple Series with three and four values each.

In [ ]:
s1 = pd.Series(index=['apple', 'banana', 'coconut'], data=[5, 20, 2])
s2 = pd.Series(index=['apple', 'banana', 'dragonfruit', 'grape'], data=[9, 8, 11, 4])
s1

In [ ]:
s2

As with DataFrames, the `pd.concat` function stack the objects vertically by default.

In [ ]:
pd.concat([s1, s2])

Each Series may be labeled with the `keys` parameter with the resulting levels named with the `names` parameter. Here, we use `names` to label both levels.

In [ ]:
pd.concat([s1, s2], keys=['Niko', 'Penelope'], names=['Name', 'Fruit'])

If using a dictionary, the keys will automatically be used as the labels and placed in a new index level.

In [ ]:
fruit_dict = {'Niko': s1, 'Penelope': s2}
pd.concat(fruit_dict)

### Concatenating Series horizontally

Set `axis` to 1 to concatenate horizontally which has the effect of making each Series as a column in a DataFrame. Automatic alignment of the index takes place here with missing values appearing where misalignments occur. The new column names will be the keys of the dictionary.

In [ ]:
pd.concat(fruit_dict, axis=1)

## Appending rows to DataFrames

Closely related to the `pd.concat` function is the DataFrame `append` method, which *only appends rows and NOT columns* to an existing DataFrame. Its first argument can either be a single or list of DataFrames and will automatically align by the columns.

In [ ]:
amzn.append(aapl)

When there is a misalignment, an outer join takes place just like `pd.concat`. Here, we use `amzn2` which uses the column name `'close'` instead of `'adjusted_close'`.

In [ ]:
amzn2.append(aapl)

Here, we pass in a list of DataFrames to append.

In [ ]:
amzn.append([aapl, amzn])

In rare instances, you might need to append a single new row to your DataFrame by hand. You can do this by creating a Series using a dictionary with the column names as keys. You must also set the `name` parameter of the Series constructor. Here, we set it to a datetime to match the data type of the current DataFrame.

In [ ]:
new_row = pd.Series({'adjusted_close': 999.99, 'volume': 9999999, 'symbol': 'tsla'},
                   name=pd.Timestamp('2020-08-01'))
amzn.append(new_row)

I would not recommend using the above syntax to manually add many new rows (i.e. within a for-loop) as it is quite inefficient to do so. First seek out vectorized operations that work on all of the data. Appending single rows like this should only really be done when there are no other options to do so.

## Adding new columns with `assign`

We've already learned how to add new columns with the *just the brackets* operator using an assignment statement. i.e. `df['new col'] = df['old col'] + 5`. Doing so permanently alters the DataFrame. One limitation of this operation is that it cannot be completed within a chain of methods. i.e. `df.drop_duplicates()['new col'] = 5`

As an alternative, the `assign` method exists to add new columns as a method. It always returns a **copy** of the data, leaving the original DataFrame unchanged. We'll work with a few of the rows and columns of the airbnb dataset to showcase the `assign` method.

In [ ]:
usecols = ['neighborhood', 'accommodates', 'security_deposit', 
           'cleaning_fee', 'price']
airbnb = pd.read_csv('../data/airbnb.csv', usecols=usecols).head(3)
airbnb

The `assign` method has no pre-defined keyword arguments. In order to use it, you must supply it at least one keyword argument. The name of the argument will be the new column name with its value as the new column values. Below is the generic syntax for adding three new columns.

```python
df.assign(new_col1=value1, new_col2=value2, new_col3=value3)
```
The new value can be one of the following:

* a scalar value
* a sequence of values the same length as the DataFrame
* a function

### Using `assign` to set a column to be all the same values

Let's begin by appending a new column with all the same values. Here, we set the keyword argument, `new_col`, to the scalar value -99.

In [ ]:
airbnb.assign(new_col1=-99)

As the `assign` method produces a copy of the data, you'll have to assign the result to a new variable in in order to continue using it. We verify that the original DataFrame remains unchanged.

In [ ]:
airbnb

### Using `assign` to set a column to be different values

A sequence of values may also be used. Here, we calculate the price per person as Series and use `assign` to create the new column.

In [ ]:
ppp = airbnb['price'] / airbnb['accommodates']
airbnb.assign(price_per_person=ppp)

### Using `assign` with functions to add new columns

You can set your keyword arguments to a custom function, which will implicitly be passed the entire DataFrame as its only argument. Here, we create three new columns with anonymous functions using the variable `df` to refer to the DataFrame. The `assign` method allows you to reference columns that you created within `assign` itself as long as they appear after it in the keyword argument list. The `fees_per_person` column references `total_fees` which was created in the same call to `assign`.

In [ ]:
airbnb.assign(price_per_person=lambda df: df['price'] / df['accommodates'], 
              total_fees=lambda df: df['security_deposit'] + df['cleaning_fee'],
              fees_per_person=lambda df: df['total_fees'] / df['accommodates'])

### Column names with spaces

In the above calls to `assign`, column names are limited by the rules that Python has for parameter names. This means that they cannot contain spaces, start with numbers, or have operators within them. To work around this limitation, you can wrap the column names and their new values within a dictionary and unpack them within the method call with `**`.

In [ ]:
kwargs = {'Price per Person': lambda df: df['price'] / df['accommodates'], 
          'Total Fees': lambda df: df['security_deposit'] + df['cleaning_fee'],
          'Fees per Person': lambda df: df['Total Fees'] / df['accommodates']}
airbnb.assign(**kwargs)

## Exercises

### Exercise 1

<span style="color:green; font-size:16px">Read in all of the files in the `../data/weather` directory except `city_attributes.csv` as DataFrames placing the `datetime` column in the index. Store the data in a dictionary using the file name (without extension) as the key. Concatenate the DataFrames vertically labeling each DataFrame in the index appropriately.</span>

### Exercise 2

<span style="color:green; font-size:16px">Use the dictionary created in Exercise 1 to concatenate the DataFrames horizontally.</span>

### Exercise 3

<span style="color:green; font-size:16px">Write a function that accepts the dictionary created from Exercise 1 and a city name. Return a DataFrame of all of the weather metrics for that city for each day. The index will be the datetime and the columns will be each individual weather metric.</span>

### Exercise 4

<span style="color:green; font-size:16px">Iterate through all of the DataFrames in the dictionary created in Exercise 1 and use the `assign` method to add a column for the `weather_type`. Save the results in a **list** named `dfs_list`.</span>

### Exercise 5

<span style="color:green; font-size:16px">Concatenate all of the DataFrames in `dfs_list` vertically using the `append` method. Create a single DataFrame.</span>

### Exercise 6

<span style="color:green; font-size:16px">Select the temperature data from the dictionary created in Exercise 1. Find the mean, median, min, and max temperature per month for the city of Houston. Then create a new column named `mean_median_diff` that contains the absolute difference between the mean and median. Also create the column `min_max_diff` that rounds the absolute difference between the min and max temperatures. Use one line of code.</span>